In [1]:
# Imports

import os
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from loguru import logger
from pathlib import Path

# Local imports
sys.path.append(r"./utils")
from utils import utils

Using TensorFlow backend.


In [2]:
# Constants

SAMPLE_DATA = False
DEBUG_MODEL = False

GROUP_NAME = "ethnicity_groups"  # TODO : Changed this for ethnicity segments. 
MODEL_DICT = \
            {"NN": {'epochs': 25} , 
             "LR": {'max_iter': 100}
            }
SELF_REPORTED_COLS = \
           ['age',
            # 'country_canada', 
            # 'country_united kingdom',
            # 'country_united states',
            # 'database_dating',
            # 'database_fb',
            # 'gender',
           ]
DATA_DROP_COLS = \
           ['Unnamed: 0', # index columns
            # 'pol',  # label column
            'gender', # self reported and filtered already
            # 'age', # self-reported  
            'country', # self reported and filtered already
            'userid', # index equivalent column 
            'pol_dat_us', # redundant columns with label
            'pol_dat_ca', # redundant columns with label
            'pol_dat_uk', # redundant columns with label
            'pol_fb_us', # redundant columns with label
            'database', # filtered already 
            'ethnicity.value' # filtered already
            ]
RESULTS_COLS = \
            ["Group Name", 
             "Model", 
             "Feature Set", 
             "Test AUC", 
             "Test Accuracy",
            ]
DATA_DIR = "./data/full/"
RESULTS_DIR = f"./results/full/{GROUP_NAME}/" #TODO: ensure that the folder exists
RESULTS_STATS_FILENAME = GROUP_NAME + '.csv'
RESULTS_MODEL_FILENAME_PREFIX = GROUP_NAME

if SAMPLE_DATA:
  DATA_DIR = "./data/sample/"
  RESULTS_DIR = f"./results/sample/{GROUP_NAME}/"
  ASSERT_DATA_SHAPE_0 = 31742
  ASSERT_DATA_SHAPE_1 = 2092
  DATA_DROP_COLS = DATA_DROP_COLS \
                  + ['Unnamed: 0.1'] #TODO: Regenerate sample with index=False and remove this

if DEBUG_MODEL:
  MODEL_DICT = \
            {"NN": {'epochs': 1} , 
             "LR": {'max_iter': 1}
            }

logger.debug(f"Started the script for {GROUP_NAME}.")

2021-08-18 17:22:49.202 | DEBUG    | __main__:<module>:60 - Started the script for ethnicity_groups.


In [3]:
# Read datasets

folders = os.listdir(DATA_DIR)
dataset_paths = []
for folder in tqdm(folders):
  logger.debug(f"In folder {folder}.")
  csv_files = os.listdir(DATA_DIR + folder)
  for csv in csv_files:
    if '.csv' in csv:
      logger.debug(DATA_DIR + folder + "/" + csv)
      dataset_paths.append(DATA_DIR + folder + "/" + csv)


  0%|          | 0/9 [00:00<?, ?it/s]2021-08-18 17:22:53.243 | DEBUG    | __main__:<module>:6 - In folder UK_0_dating.
2021-08-18 17:22:53.244 | DEBUG    | __main__:<module>:10 - ./data/sample/UK_0_dating/segment_united kingdom_0_dating_black.csv
2021-08-18 17:22:53.245 | DEBUG    | __main__:<module>:10 - ./data/sample/UK_0_dating/segment_united kingdom_0_dating_asian.csv
2021-08-18 17:22:53.246 | DEBUG    | __main__:<module>:10 - ./data/sample/UK_0_dating/segment_united kingdom_0_dating_india.csv
2021-08-18 17:22:53.246 | DEBUG    | __main__:<module>:10 - ./data/sample/UK_0_dating/segment_united kingdom_0_dating_white.csv
2021-08-18 17:22:53.247 | DEBUG    | __main__:<module>:6 - In folder Canada_1_dating.
2021-08-18 17:22:53.248 | DEBUG    | __main__:<module>:10 - ./data/sample/Canada_1_dating/segment_canada_1_dating_asian.csv
2021-08-18 17:22:53.249 | DEBUG    | __main__:<module>:10 - ./data/sample/Canada_1_dating/segment_canada_1_dating_india.csv
2021-08-18 17:22:53.249 | DEBUG    

In [4]:
results = []

for dataset_path in dataset_paths:
  data = pd.read_csv(dataset_path)
  
  # Clean the data
  data = data.drop(DATA_DROP_COLS, axis=1)
  data = utils.get_clean_data(data)
  logger.debug(f"Data size after cleaning is {data.shape}")
  data_y = data['pol'].replace({"liberal": 1, "conservative": 0})
  data = data.drop('pol', axis = 1)
  all_features = data.columns

  # Split the data
  X_train, X_test, y_train, y_test = train_test_split(data, data_y, test_size = 0.2, random_state = 2021) 
  del data, data_y

  ## Define features for the various settings
  image_cols = list(map(str, range(1, 2049)))
  image_and_self_reported_cols = image_cols + SELF_REPORTED_COLS
  image_and_extracted_cols = [x for x in all_features if x not in SELF_REPORTED_COLS]
  image_and_self_reported_and_extracted_cols = all_features

  data_dict = {
              "Image Features" : image_cols,
              "Image and Self Reported Features" : image_and_self_reported_cols,
              "Image and Extracted Features": image_and_extracted_cols,
              "Image, Self-reported and Extracted Features": image_and_self_reported_and_extracted_cols
              }

  # which group it is being processed on 
  SEGMENT_NAME = utils.get_dataframe_name(dataset_path)
  logger.debug(f"Started the script for {SEGMENT_NAME}.")

  # Fit models and log results
  for data_name, data_set_features in tqdm(data_dict.items()):
    for model_name, model_params in MODEL_DICT.items():
      try:
        save_model_filepath = Path(RESULTS_DIR \
                        + SEGMENT_NAME\
                        + "_" + model_name \
                        + "_" + data_name.replace(" ","_").replace(",","").replace("-","_") \
                        + '.mdl')

        if save_model_filepath.is_file() or save_model_filepath.is_dir():
          logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model already exists.")
          model = utils.read_model(model_name, save_model_filepath)
          logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model read from disk.")
          
        else:
          logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model training started.")
          model = utils.fit_model(model_name,
                                X_train[data_set_features],
                                y_train,
                                model_params = model_params)

          utils.save_model(model, model_name, save_model_filepath)
          logger.debug(f"{GROUP_NAME}, {model_name}, {data_name}: model training ended and model saved.")
        
        auc, acc = utils.get_metrics(model_name,
                                     model,
                                     X_test[data_set_features],
                                     y_test)
                                     
        results.append([GROUP_NAME, model_name, data_name, auc, acc])
        logger.debug(f"{GROUP_NAME}, {model_name}, {data_name}: model training ended. AUC: {auc}, accuracy: {acc}")
      
      except Exception as error:
        logger.exception(error)
        logger.error(f"{SEGMENT_NAME}, {model_name}, {data_name}: Error occured!")

2021-08-18 17:23:00.416 | DEBUG    | __main__:<module>:9 - Data size after cleaning is (1000, 2076)
2021-08-18 17:23:00.424 | DEBUG    | __main__:<module>:33 - Started the script for united kingdom_0_dating_black.
  0%|          | 0/4 [00:00<?, ?it/s]2021-08-18 17:23:00.425 | DEBUG    | __main__:<module>:46 - united kingdom_0_dating_black, NN, Image Features: model already exists.
2021-08-18 17:23:00.807 | DEBUG    | __main__:<module>:48 - united kingdom_0_dating_black, NN, Image Features: model read from disk.
2021-08-18 17:23:01.134 | DEBUG    | __main__:<module>:66 - ethnicity_groups, NN, Image Features: model training ended. AUC: 58.24, accuracy: 56.5
2021-08-18 17:23:01.135 | DEBUG    | __main__:<module>:46 - united kingdom_0_dating_black, LR, Image Features: model already exists.
2021-08-18 17:23:01.136 | DEBUG    | __main__:<module>:48 - united kingdom_0_dating_black, LR, Image Features: model read from disk.
2021-08-18 17:23:01.166 | DEBUG    | __main__:<module>:66 - ethnicity_

In [5]:
# Save results summary to disk

save_results_filepath = Path(RESULTS_DIR + RESULTS_STATS_FILENAME)
utils.save_results(results_array = results, 
                     location = save_results_filepath,
                     columns = RESULTS_COLS)
print(pd.DataFrame(results,columns = RESULTS_COLS))
logger.debug(f"Script for {GROUP_NAME} finished.")

2021-08-18 17:24:38.778 | DEBUG    | utils.utils:save_results:263 - Results Saved.
2021-08-18 17:24:38.783 | DEBUG    | __main__:<module>:8 - Script for ethnicity_groups finished.


           Group Name Model                                  Feature Set  \
0    ethnicity_groups    NN                               Image Features   
1    ethnicity_groups    LR                               Image Features   
2    ethnicity_groups    NN             Image and Self Reported Features   
3    ethnicity_groups    LR             Image and Self Reported Features   
4    ethnicity_groups    NN                 Image and Extracted Features   
..                ...   ...                                          ...   
249  ethnicity_groups    LR             Image and Self Reported Features   
250  ethnicity_groups    NN                 Image and Extracted Features   
251  ethnicity_groups    LR                 Image and Extracted Features   
252  ethnicity_groups    NN  Image, Self-reported and Extracted Features   
253  ethnicity_groups    LR  Image, Self-reported and Extracted Features   

     Test AUC  Test Accuracy  
0       58.24           56.5  
1       80.90           7